# NetworKit Randomization Tutorial

The randomization module implements algorithms to perturb existing graphs. This is commonly used to obtain a null-model for hypothesis testing in network analysis (see below for an example). Intuitively one tests whether an observation in original also appears in *similar* graphs. By doing so one can quantify the statistical significance of the observation. The ensemble of *similar* networks is commonly chosen as the set of (simple) graphs that have the same degree sequence. 

The Curveball algorithms creates a random sample with the same degree sequence, by repeatedly selecting two random nodes and randomly exchanging their neighbourhoods (this is referred to as a trade). This corresponds to a random walk on the space of all graphs in the ensemble. The more trades are carried out, the less correlated input and output are. While there a no practical lower bounds, one typically choose 10 to 100 times the number n of nodes.

GlobalCurveball is a related algorithm that is typically faster than Curveball. Also it's implementation is more versatile (e.g., it support directed and undirected graphs). In each step it carries out n/2 trades involving all nodes. A typical choice of the number of GlobalTrades is 10 to 100. **We recommend the usage of GlobalCurveball over Curveball for performance reasons**.

In [ ]:
import networkit as nk

## Global Curveball

The Global Curveball class is an implementation of EM-GCB proposed in `Parallel and I/O-efficient Randomisation of Massive Networks using Global Curveball Trades", Carstens et al., ESA 2018`. The algorithm perturbs an input graph, by iteratively randomizing the neighbourhoods of node pairs. For a large number of global trades this process is shown to produce a uniform sample from the set of all graphs with the same degree sequence as the input graph.

The `GlobalCurveball(G, number_of_global_rounds=20, allowSelfLoops=False, degreePreservingShufflePreprocessing=True)` constructor expecs a graph, followed by the parameter `number_of_global_rounds` which dictactes the number of global rounds to carry out. To accelerate the perturbation process we recommend to set degreePreservingShufflePreprocessing to true. This jump-starts the randomization process and yields faster convergence of the algorithm (see "Smaller Universes for Uniform Sampling of 0,1-matrices with fixed row and column sums" * by Annabell Berger, Corrie Jacobien Carstens [https://arxiv.org/abs/1803.02624]).
For directed simple graphs the algorithm will not yield a uniform sample unless the preprocessing is enabled.

In [ ]:
# Read graph
G = nk.graphio.readGraph("../input/karate.graph", nk.Format.METIS)
print(G.numberOfNodes(), G.numberOfEdges())

In [ ]:
# Initialize algorithm
globalCurve = nk.randomization.GlobalCurveball(G)

In [ ]:
# Run algorithm
globalCurve.run()

In [ ]:
# Get randomized graph
randomGlobalG = globalCurve.getGraph()

In [ ]:
# Verify
print(randomGlobalG.numberOfNodes(), randomGlobalG.numberOfEdges())
assert(randomGlobalG.numberOfNodes() == G.numberOfNodes())
for u in range (randomGlobalG.upperNodeIdBound()):
    assert(randomGlobalG.degree(u) == G.degree(u))

## Curveball

The Curveball class in an implementation of IM-CB proposed in `Parallel and I/O-efficient Randomisation of Massive Networks using Global Curveball Trades", Carstens et al., ESA 2018`. The algorithm perturbs an undirected and unweighted input graph, by iteratively randomizing the neighbourhoods of node pairs.

The `Curveball(G)` constructor expects an undirected, unweighted graph. This class does not support the `run()` method; it instead provides the `run(trades)` method. The parameter `trades` is a vector of pairs of nodes. The `run` method can be called multiple times.

Following is an example:

In [ ]:
# Read graph
G = nk.graphio.readGraph("../input/celegans_metabolic.thrill", nk.Format.ThrillBinary)
print(G.numberOfNodes(), G.numberOfEdges())

In [ ]:
# Initialize algorithm
curve = nk.randomization.Curveball(G)

The `CurveballUniformTradeGenerator(num_trades, num_nodes)` class can be used to generate trades. `num_trades` is the number of trades to generate while `num_nodes` is the number of node indices to draw from. It generates a trade sequence consisting of `num_trades` many single trades. Each trade contains two different node indices drawn uniformly at random from the interval [0, num_nodes).

In [ ]:
# Generate trade sequence
utg = nk.randomization.CurveballUniformTradeGenerator(G.numberOfNodes(), G.numberOfNodes())
trades = utg.generate()

In [ ]:
# Run algorithm
curve.run(trades)

In [ ]:
# Get randomized graph
randomCurveG = curve.getGraph()

In [ ]:
# Verify
print(randomCurveG.numberOfNodes(), randomCurveG.numberOfEdges())
assert(randomCurveG.numberOfNodes() == G.numberOfNodes())
for u in range (randomCurveG.upperNodeIdBound()):
    assert(randomCurveG.degree(u) == G.degree(u))

## DegreePreservingShuffle

DegreePreservingShuffle is available as a standalone module. It will relabel nodes while keeping their degrees but wont change the topology of the graph. The constructor `DegreePreservingShuffle(G)` expects a directed or undirected graph.

In [ ]:
# Generate graph
G = nk.generators.ErdosRenyiGenerator(200, 0.2).generate()

In [ ]:
# Initialze algorithm
dps = nk.randomization.DegreePreservingShuffle(G)

In [ ]:
# Run algorithm
dps.run()

In [ ]:
randomG = dps.getGraph()

In [ ]:
# Verify
for u in range(G.upperNodeIdBound()):
    assert(G.degree(u) == randomG.degree(u))

## Application example

Our hypothesis is that Hyperbolic Graphs (RHGs) have a high clustering. To test the hypothesis
we first obtain an RHG and compute its local clustering coefficient. Then, we compute five random graphs with the same degree sequence and observe that their mean LCC score is much lower.
This gives empirical support towards our hypothesis.

In [ ]:
def llcScore(G):
    """Compute average local clustering coefficient"""
    return sum(nk.centrality.LocalClusteringCoefficient(G).run().scores()) / G.numberOfNodes()

# Hyperbolic graphs are known to have an above average clustering
G = nk.generators.HyperbolicGenerator(1000, 10).generate()
print("Avg. clustering of input:         %.3f" % llcScore(G))

for i in range(5):
    # Take 5 random graphs with the same degree sequence
    sampledGraph = nk.randomization.GlobalCurveball(G).run().getGraph()
    print("Avg. clustering of random sample: %.3f" % llcScore(sampledGraph))